In [1]:
import pandas as pd
import os

# Ruta a la carpeta donde se encuentran los archivos CSV
folder_path = os.getcwd() # Esto usa el directorio de trabajo actual

# Crear una lista para almacenar los DataFrames
all_data = []

# Iterar sobre todos los archivos CSV en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Cargar el archivo CSV
        file_path = os.path.join(folder_path, filename)
        data = pd.read_csv(file_path)
        
        # Añadir el DataFrame a la lista
        all_data.append(data)

# Unir todos los DataFrames en uno solo
combined_data = pd.concat(all_data, ignore_index=True)

# Mostrar las primeras filas del DataFrame combinado
print(combined_data.head())


            ride_id  rideable_type           started_at             ended_at  \
0  4EAD8F1AD547356B  electric_bike  2023-11-30 21:50:05  2023-11-30 22:13:27   
1  6322270563BF5470  electric_bike  2023-11-03 09:44:02  2023-11-03 10:17:15   
2  B37BDE091ECA38E0  electric_bike  2023-11-30 11:39:44  2023-11-30 11:40:08   
3  CF0CA5DD26E4F90E   classic_bike  2023-11-08 10:01:45  2023-11-08 10:27:05   
4  EB8381AA641348DB   classic_bike  2023-11-03 16:20:25  2023-11-03 16:54:25   

       start_station_name start_station_id               end_station_name  \
0         Millennium Park            13008  Pine Grove Ave & Waveland Ave   
1  Broadway & Sheridan Rd            13323         Broadway & Sheridan Rd   
2   State St & Pearson St     TA1307000061          State St & Pearson St   
3     Theater on the Lake     TA1308000001            Theater on the Lake   
4     Theater on the Lake     TA1308000001            Theater on the Lake   

  end_station_id  start_lat  start_lng    end_lat    end

# Ahora iniciamos con el proceso de limpieza


Necesito eliminar los registros no valiidos de columans que me van a ser utiles:  ride_id, started_at, ended_at, member_casual

Así como eliminar los registros con una duración de viaje inválida

In [3]:
# Eliminar registros con valores nulos en columnas clave
combined_data = combined_data.dropna(subset=['ride_id', 'started_at', 'ended_at', 'member_casual'])

In [5]:
# Convertir las columnas de fecha a tipo datetime
combined_data['started_at'] = pd.to_datetime(combined_data['started_at'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
combined_data['ended_at'] = pd.to_datetime(combined_data['ended_at'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

# Revisar si hay valores nulos después de la conversión
print(combined_data[['started_at', 'ended_at']].isnull().sum())

started_at    7305758
ended_at      7305758
dtype: int64


In [6]:
# Calcular la duración del viaje en minutos
combined_data['ride_length'] = (combined_data['ended_at'] - combined_data['started_at']).dt.total_seconds() / 60

In [7]:
# Eliminar registros con duración de viaje menor o igual a 0
combined_data = combined_data[combined_data['ride_length'] > 0]

In [ ]:
# Verificar los cambios
print(combined_data.isnull().sum()) # Verificar si hay valores nulos

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    721926
start_station_id      721926
end_station_name      759146
end_station_id        759146
start_lat                  0
start_lng                  0
end_lat                 5708
end_lng                 5708
member_casual              0
ride_length                0
dtype: int64


In [9]:
print(combined_data['ride_length'].describe())  # Verificar estadísticas de la duración del viaje

count    4.559898e+06
mean     1.613407e+01
std      6.275758e+01
min      1.666667e-02
25%      5.100000e+00
50%      8.816667e+00
75%      1.563333e+01
max      1.559933e+03
Name: ride_length, dtype: float64


In [10]:
# Calcular el porcentaje de valores nulos en cada columna
null_percentage = combined_data.isnull().mean() * 100

# Mostrar el porcentaje de valores nulos
print(null_percentage)

ride_id                0.000000
rideable_type          0.000000
started_at             0.000000
ended_at               0.000000
start_station_name    15.832065
start_station_id      15.832065
end_station_name      16.648311
end_station_id        16.648311
start_lat              0.000000
start_lng              0.000000
end_lat                0.125178
end_lng                0.125178
member_casual          0.000000
ride_length            0.000000
dtype: float64


Debido a que los porcentajes de datos nulos que representa en las columnas de nombres e ids de las estaciones es de 16%-17% se decidio remplazar los nulos por el termino **Desconocido** 

Y en el caso de las columnas con la latitud y longuitud se decidio imputar con los valores de la media de las estaciones de finalización, considerando que esto no significaria un gran impacto en los datos, ya que representan unicamrnte el 0.12%

In [17]:
# Imputar valores nulos para las estaciones con un valor de "Desconocido"
combined_data['start_station_name'].fillna('Desconocido', inplace=True)
combined_data['end_station_name'].fillna('Desconocido', inplace=True)
combined_data['start_station_id'].fillna('Desconocido', inplace=True)
combined_data['end_station_id'].fillna('Desconocido', inplace=True)

# Imputar valores nulos en las coordenadas con la media de las estaciones de finalización
combined_data['end_lat'].fillna(combined_data['end_lat'].mean(), inplace=True)
combined_data['end_lng'].fillna(combined_data['end_lng'].mean(), inplace=True)


C:\Users\Luis Méndez\AppData\Local\Temp\ipykernel_16776\1510154459.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_data['start_station_id'].fillna('Desconocido', inplace=True)
C:\Users\Luis Méndez\AppData\Local\Temp\ipykernel_16776\1510154459.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

### Agrego algunas columnas derivadas que me serán utiles en el analisis, como el dia de la semana y la hora del día, esto me ayuda a ampliar la busqueda de patrones de uso 

In [18]:
# Agregar columna de día de la semana
combined_data['day_of_week'] = combined_data['started_at'].dt.day_name()

In [19]:
# Agregar columna de hora del día
combined_data['time_of_day'] = combined_data['started_at'].dt.hour.apply(
    lambda x: 'Morning' if 6 <= x < 12 else ('Afternoon' if 12 <= x < 18 else ('Evening' if 18 <= x < 24 else 'Night'))
)

In [20]:
# Verificar las nuevas columnas
print(combined_data[['started_at', 'day_of_week', 'time_of_day']].head())

           started_at day_of_week time_of_day
0 2023-11-30 21:50:05    Thursday     Evening
1 2023-11-03 09:44:02      Friday     Morning
2 2023-11-30 11:39:44    Thursday     Morning
3 2023-11-08 10:01:45   Wednesday     Morning
4 2023-11-03 16:20:25      Friday   Afternoon


### Verificación de la calidad de los datos

In [21]:
# Revisar los primeros registros para asegurarse de que las transformaciones se hicieron correctamente
print(combined_data.head())

            ride_id  rideable_type          started_at            ended_at  \
0  4EAD8F1AD547356B  electric_bike 2023-11-30 21:50:05 2023-11-30 22:13:27   
1  6322270563BF5470  electric_bike 2023-11-03 09:44:02 2023-11-03 10:17:15   
2  B37BDE091ECA38E0  electric_bike 2023-11-30 11:39:44 2023-11-30 11:40:08   
3  CF0CA5DD26E4F90E   classic_bike 2023-11-08 10:01:45 2023-11-08 10:27:05   
4  EB8381AA641348DB   classic_bike 2023-11-03 16:20:25 2023-11-03 16:54:25   

       start_station_name start_station_id               end_station_name  \
0         Millennium Park            13008  Pine Grove Ave & Waveland Ave   
1  Broadway & Sheridan Rd            13323         Broadway & Sheridan Rd   
2   State St & Pearson St     TA1307000061          State St & Pearson St   
3     Theater on the Lake     TA1308000001            Theater on the Lake   
4     Theater on the Lake     TA1308000001            Theater on the Lake   

  end_station_id  start_lat  start_lng    end_lat    end_lng member_

In [22]:
# Verificar cuántos valores nulos quedan
print(combined_data.isnull().sum())

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
ride_length           0
day_of_week           0
time_of_day           0
dtype: int64


### Guardar mi Dataframe limpio como un archivo CSV

In [24]:
# Guardar el DataFrame limpio en un nuevo archivo CSV
combined_data.to_csv('cleaned_combined_data.csv', index=False)
